In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('default')

from sklearn import datasets
from sklearn.model_selection import train_test_split

# Naive Bayes
Is a supervised learning algorithm based on applying Bayes’ theorem with the “naive” assumption of conditional independence between every pair of features given the value of the class variable.
The different naive Bayes classifiers differ mainly by the assumptions they make regarding the distribution of $P(x_i|y)$.

They work quite well, can be very fast and require a small amount of training data to estimate the necessary parameters. They have been famously used in document classification and spam filtering. However, it is known to be a bad estimator, so the probability outputs from are not to be taken too seriously.

Bayes Theorem:  

$P(A|B) = \frac{P(B|A) * P(A)} {P(B)} $   

The probability of A given B.    

In case of our target and data:    

$P(y|X)$   - Posterior Probability    
$P(y)$     - Prior probability of y    
$P(x_i|y)$ - Class conditional probability

Using the chain rule we get:
$P(y|X) = \frac{P(y)  *  \prod_i{P(x_i|y)}} {P(X)} $

$argmax(P(y|X))$ select the class with the highest probability.

Finally: $argmax( log(P(x_1|y)) + log(P(x_2|y)) + ... + log(P(y)) )$



Using the Gaussion distribution:

$P(x_i|y) = \frac{1}{\sqrt{2 \pi \sigma_y^2}} * exp(-\frac{(x_i-\mu_y)^2}{2 \sigma_y^2}) $